# End-to-End Deployment Project

This notebook walks through a compact **end-to-end ML deployment project** — from data → training → packaging → API → Docker → CI → monitoring. It's a template you can copy into a real repo and adapt for larger projects.

The example uses the **Iris** dataset and demonstrates pragmatic steps and files you would add to a GitHub repository for productionizing a small model.

## 🎯 Objectives

- Train a simple model and save it as an artifact.
- Create a lightweight FastAPI app to serve predictions.
- Provide a `Dockerfile` to containerize the API.
- Show a sample GitHub Actions workflow for CI (train + test + build image).
- Outline basic monitoring and rollback recommendations.

## ⚙️ Project structure (suggested)

```
end-to-end-deploy/
├── data/                # optional: raw or sample data
├── model/               # saved model artifacts
│   └── iris_model.joblib
├── api/
│   ├── app.py           # FastAPI app
│   └── requirements.txt
├── Dockerfile
├── .github/workflows/ci-cd.yml
└── notebooks/
    └── 11-End_to_End_Deployment_Project.ipynb
```

## 1 — Train & save a simple model (Iris)
This code trains a RandomForest and saves it to `model/iris_model.joblib`. Run locally or as part of CI to produce the artifact.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import os

os.makedirs('model', exist_ok=True)

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Trained RandomForest — test accuracy: {acc:.4f}")

# Save model artifact
joblib.dump(clf, 'model/iris_model.joblib')
print('Model saved to model/iris_model.joblib')

## 2 — FastAPI app (`api/app.py`)

Below is a minimal FastAPI application to load the saved model and expose a `/predict` endpoint. Save as `api/app.py` in your repo.

In [ ]:
%%writefile api/app.py
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np
import os

MODEL_PATH = os.path.join(os.path.dirname(__file__), '..', 'model', 'iris_model.joblib')
MODEL_PATH = os.path.abspath(MODEL_PATH)

app = FastAPI(title='Iris Classifier')

class Features(BaseModel):
    features: list

@app.on_event('startup')
def load_model():
    global model
    model = joblib.load(MODEL_PATH)

@app.get('/')
def root():
    return {'message': 'Iris classifier up. POST to /predict'}

@app.post('/predict')
def predict(data: Features):
    arr = np.array(data.features).reshape(1, -1)
    pred = model.predict(arr)
    return {'prediction': int(pred[0])}


Also add a simple `api/requirements.txt`:

In [ ]:
%%writefile api/requirements.txt
fastapi
uvicorn[standard]
joblib
scikit-learn
numpy


## 3 — Dockerfile (root)

A concise Dockerfile to build the API image. Save as `Dockerfile` at repo root.

In [ ]:
%%writefile Dockerfile
FROM python:3.10-slim
WORKDIR /app

COPY api/ ./api/
COPY model/ ./model/
COPY --chown=root:root api/requirements.txt ./requirements.txt

RUN pip install --no-cache-dir -r requirements.txt

EXPOSE 8000
CMD ["uvicorn", "api.app:app", "--host", "0.0.0.0", "--port", "8000"]


## 4 — GitHub Actions (CI) — `.github/workflows/ci-cd.yml`

Sample pipeline that trains model, runs tests, builds Docker image (simulated), and pushes when on `main`.
Place this YAML at `.github/workflows/ci-cd.yml`.

In [ ]:
%%writefile .github/workflows/ci-cd.yml
name: CI-CD pipeline

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build-and-test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.10'
      - name: Install deps
        run: |
          pip install -r api/requirements.txt
      - name: Train model (artifact)
        run: |
          python -c "from sklearn.datasets import load_iris; from sklearn.ensemble import RandomForestClassifier; import joblib; X,y=load_iris(return_X_y=True); clf=RandomForestClassifier(); clf.fit(X,y); joblib.dump(clf,'model/iris_model.joblib')"
      - name: Basic smoke test
        run: |
          python -c "import joblib; m=joblib.load('model/iris_model.joblib'); import numpy as np; print(m.predict(np.array([ [5.1,3.5,1.4,0.2] ])))"

  build-image:
    needs: build-and-test
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main'
    steps:
      - uses: actions/checkout@v3
      - name: Build Docker image
        run: echo 'Build Docker image step (configure with DockerHub / ECR credentials)'


## 5 — How to run locally

1. Train & save model (or use the saved artifact in `model/`).
```bash
python -c "from sklearn.datasets import load_iris; from sklearn.ensemble import RandomForestClassifier; import joblib; X,y=load_iris(return_X_y=True); clf=RandomForestClassifier(); clf.fit(X,y); joblib.dump(clf,'model/iris_model.joblib')"
```
2. Start API locally (inside repo root):
```bash
pip install -r api/requirements.txt
uvicorn api.app:app --reload --port 8000
```
3. Test prediction:
```bash
curl -X POST 'http://127.0.0.1:8000/predict' -H 'Content-Type: application/json' -d '{"features":[5.1,3.5,1.4,0.2]}'
```

## 6 — Monitoring & Observability (high-level)

- **Metrics to collect:** request latency, request rates, prediction distribution, input feature statistics, model accuracy (if labels available).
- **Tools:** Prometheus + Grafana for metrics, ELK for logs, Evidently or WhyLabs for drift detection, MLflow for model registry.
- **Alerts:** trigger retraining when model accuracy falls below threshold or PSI/KS drift exceeds threshold.
- **Logging:** add structured logs in the API (request id, timestamp, features, prediction) and ship to a log store.

Example (pseudo) — push a metric to Prometheus client inside the API:
```python
from prometheus_client import Summary
REQUEST_LATENCY = Summary('request_latency_seconds', 'Latency of prediction')

@REQUEST_LATENCY.time()
def predict(...):
    ...
```


## 7 — Rollback & Canary strategy (notes)

- **Canary deployments:** deploy new version to small % of traffic and monitor metrics before full rollout.
- **Blue/Green:** keep previous version running and switch traffic when new version is validated.
- **Rollback:** automate rollback if key metrics degrade beyond thresholds.


## ✅ Summary & Next steps

- This template gives you a **reproducible, minimal** end-to-end deployment flow.
- Next steps to harden for production:
  - Add authentication, rate-limiting, and request validation.
  - Store models in a model registry (MLflow) and reference by version in deploys.
  - Implement automated retraining pipelines (Airflow / Prefect) triggered by drift.
  - Add tests (unit, integration, load) into CI pipeline.

If you want, I can now:
1. generate all files in a downloadable ZIP (code + Dockerfile + workflow), or
2. expand this template into a production-ready repo with Prometheus metrics and a tutorial README.

Which would you like next?